**owl_restrictions_on_inputs_outputs.ipynb**

check that OWL restrictions on inputs or outputs are not too abstract

**Documentation:** TODO

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [50]:
import sys
from rdflib import ConjunctiveGraph, Namespace
import json
from collections import OrderedDict

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
#g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('edam.ttl', format='turtle')
# g.load('EDAM_dev.owl')
g.bind('@base', Namespace('http://edamontology.org/'))
print("done!")
print(len(g))

Loading graph ...done!
38042


In [51]:
#g.serialize("edam.ttl", format="turtle")

Define SPARQL query to extract ID, term, and (if available) seealso and deprecated fields of all Topic concepts. Run the query.

**NB:** Use ``"/topic_"`` in query to avoid detection of http://edamontology.org/is_topic_of

In [35]:
q = """
SELECT DISTINCT ?id ?term ?seealso ?deprecated WHERE {
    ?id rdfs:subClassOf+ <http://edamontology.org/operation_0004> .
    ?id rdfs:label ?term .
    ?id rdfs:subClassOf ?restriction . 
    {
        ?restriction rdf:type owl:Restriction ; 
                 owl:onProperty :has_output  ; 
                 owl:someValuesFrom :data_0006 .
    } UNION { 
        ?restriction rdf:type owl:Restriction ; 
                 owl:onProperty :has_input  ; 
                 owl:someValuesFrom :data_0006 .
    }
}
"""

# Run SPARQL query and collate results
results = g.query(q)

In [36]:
print(f"{len(results)} EDAM classes with too broad input or output constraints")

for r in results :

    # Skip deprecated concepts
    #if r['deprecated'] == "true":
    #    print(f"skipping deprecated class {r['id']} ({r['term']})")
    #    continue
    
    print(f"WARNING Missing wikipedia link for class {r['id']} ({r['term']})")

#print(str(report).decode())

2 EDAM classes with too broad input or output constraints
WARNING Missing wikipedia link for class http://edamontology.org/operation_3357 (Format detection)
WARNING Missing wikipedia link for class http://edamontology.org/operation_0224 (Query and retrieval)


In [45]:
q = """
SELECT ?class ?my_id WHERE {
    
    #retrieve the last ID
    <http://edamontology.org> :next_id ?next_id .
    
    #match all IDs
    ?class rdf:type owl:Class .    
    BIND(strafter(str(?class), "_") AS ?my_id) .

    #retrieve the ones that should me lower than the last id
    FILTER (?my_id >= ?next_id)
} 
"""

# Run SPARQL query and collate results
results = g.query(q)

In [46]:
print(f"{len(results)} EDAM classes with too broad input or output constraints")

for r in results :
    print(r)

    # Skip deprecated concepts
    #if r['deprecated'] == "true":
    #    print(f"skipping deprecated class {r['id']} ({r['term']})")
    #    continue
    
    #print(f"WARNING Missing wikipedia link for class {r['id']} ({r['term']})")

#print(str(report).decode())

3 EDAM classes with too broad input or output constraints
(rdflib.term.URIRef('http://edamontology.org/data_4022'), rdflib.term.Literal('4022'))
(rdflib.term.URIRef('http://edamontology.org/topic_5000'), rdflib.term.Literal('5000'))
(rdflib.term.URIRef('http://edamontology.org/operation_4023'), rdflib.term.Literal('4023'))


In [72]:
q = """
SELECT DISTINCT ?x ?y WHERE {
    
    #?x rdfs:label ?l1 . 
    #?x rdfs:label ?l2 . 
    #FILTER (?l1 != ?l2) .
    
    ?y a owl:Class .
    FILTER NOT EXISTS {?y rdfs:label ?label}
} 
"""

# Run SPARQL query and collate results
results = g.query(q)

In [73]:
%%time
print(f"{len(results)} EDAM classes with 2 labels")

for r in results :
    print(r)

    # Skip deprecated concepts
    #if r['deprecated'] == "true":
    #    print(f"skipping deprecated class {r['id']} ({r['term']})")
    #    continue
    
    #print(f"WARNING Missing wikipedia link for class {r['id']} ({r['term']})")

#print(str(report).decode())

3 EDAM classes with 2 labels
(None, rdflib.term.URIRef('http://www.w3.org/2002/07/owl#DeprecatedClass'))
(None, rdflib.term.BNode('ub3bL36075C16'))
(None, rdflib.term.BNode('ub3bL42578C17'))
CPU times: user 553 ms, sys: 2.8 ms, total: 556 ms
Wall time: 556 ms
